In [ ]:
import pandas as pd
import numpy as np

# Número de registros a generar
num_records = 2000

# --- PASO 1: Generar características estructurales y demográficas ---
data = {
    # Variables Demográficas
    'EdadJefeHogar': np.random.randint(18, 80, size=num_records),
    'SexoJefeHogar': np.random.choice(['Hombre', 'Mujer'], size=num_records, p=[0.7, 0.3]),
    'MiembrosHogar': np.random.randint(1, 10, size=num_records),
    'RatioDependencia': np.random.uniform(0, 3, size=num_records).round(2),
    'LenguaMaterna': np.random.choice(['Español', 'Quechua', 'Aymara', 'Otra'], size=num_records, p=[0.85, 0.12, 0.02, 0.01]),
    
    # Variables Socioeconómicas y de Vivienda (Causas Estructurales)
    'NivelEducativoJefeHogar': np.random.choice(['Ninguno', 'Primaria', 'Secundaria', 'Superior'], size=num_records, p=[0.1, 0.3, 0.4, 0.2]),
    'TipoEmpleo': np.random.choice(['Formal', 'Informal', 'Desempleado'], size=num_records, p=[0.3, 0.6, 0.1]),
    'AreaResidencia': np.random.choice(['Urbana', 'Rural'], size=num_records, p=[0.75, 0.25]),
    'Region': np.random.choice(['Costa', 'Sierra', 'Selva'], size=num_records, p=[0.5, 0.3, 0.2]),
    'AccesoAguaPotable': np.random.choice([1, 0], size=num_records, p=[0.8, 0.2]),
    'AccesoSaneamiento': np.random.choice([1, 0], size=num_records, p=[0.7, 0.3]),
    'Hacinamiento': np.random.choice([1, 0], size=num_records, p=[0.3, 0.7]),
    'PoseeActivos': np.random.choice([1, 0], size=num_records, p=[0.6, 0.4]),
    
    # Otras variables de vivienda
    'TenenciaVivienda': np.random.choice(['Propia', 'Alquilada', 'Cedida'], size=num_records, p=[0.6, 0.3, 0.1]),
    'MaterialParedes': np.random.choice(['Ladrillo/Cemento', 'Adobe', 'Madera/Esteras'], size=num_records, p=[0.65, 0.25, 0.1]),
    'AccesoElectricidad': np.random.choice([1, 0], size=num_records, p=[0.9, 0.1]),
}

df = pd.DataFrame(data)

# --- PASO 2: coherencia---

# Los años de estudio deben depender del nivel educativo
def anios_por_nivel(nivel):
    if nivel == 'Ninguno': return np.random.randint(0, 1)
    if nivel == 'Primaria': return np.random.randint(1, 7)
    if nivel == 'Secundaria': return np.random.randint(7, 12)
    if nivel == 'Superior': return np.random.randint(12, 22)
df['AniosEstudioJefeHogar'] = df['NivelEducativoJefeHogar'].apply(anios_por_nivel)

# --- PASO 3: Generar el Ingreso como CONSECUENCIA de las variables estructurales ---
# El ingreso ya no es aleatorio, depende de los factores que causan la pobreza.

base_ingreso = 1000  # Ingreso base en Soles (aproximado a la RMV)

# Modificadores de ingreso
mod_educacion = df['NivelEducativoJefeHogar'].map({'Ninguno': 0.8, 'Primaria': 1.0, 'Secundaria': 1.5, 'Superior': 2.5})
mod_empleo = df['TipoEmpleo'].map({'Desempleado': 0.5, 'Informal': 1.0, 'Formal': 2.0})
mod_area = df['AreaResidencia'].map({'Rural': 0.8, 'Urbana': 1.2})
mod_region = df['Region'].map({'Selva': 0.9, 'Sierra': 1.0, 'Costa': 1.3})

# Calculamos el ingreso mensual con los modificadores y un componente aleatorio
ingreso_calculado = base_ingreso * mod_educacion * mod_empleo * mod_area * mod_region
df['IngresoMensualHogar'] = (ingreso_calculado * np.random.lognormal(mean=0.1, sigma=0.4, size=num_records)).round(2)

# El gasto sigue siendo una función del ingreso
df['GastoMensualHogar'] = (df['IngresoMensualHogar'] * np.random.uniform(0.7, 1.2, size=num_records)).round(2)

# --- PASO 4: Crear la Variable Objetivo (PobrezaMonetaria) basada en una línea de pobreza ---


# Línea de pobreza per cápita . 
# La ajustamos por el tamaño del hogar.
linea_pobreza_per_capita = 400 
df['LineaPobrezaHogar'] = linea_pobreza_per_capita * df['MiembrosHogar']

# Un hogar es pobre si su ingreso está por debajo de la línea de pobreza del hogar.
df['PobrezaMonetaria'] = (df['IngresoMensualHogar'] < df['LineaPobrezaHogar']).astype(int)

# Eliminamos la columna auxiliar
df = df.drop(columns=['LineaPobrezaHogar'])


# Reordenar columnas para tener la variable objetivo al final
columnas = [col for col in df.columns if col != 'PobrezaMonetaria'] + ['PobrezaMonetaria']
df = df[columnas]

# Guardar en archivo CSV
df.to_csv('prediccion_pobreza_peru.csv', index=False)

print("Base de datos 'prediccion_pobreza_peru.csv' creada exitosamente con una lógica causal mejorada.")
print(df.head())
print("\nDescripción de la base de datos:")
print(df.info())
print("\nDistribución de la nueva variable de pobreza:")
print(df['PobrezaMonetaria'].value_counts(normalize=True))